In [1]:
import pandas as pd
import datetime
import numpy as np
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import pickle

In [2]:
train1 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_1.csv')
train2 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_2.csv')
train = pd.concat([train1, train2])
train = train[train.notnull().apply(all, axis=1)]
dev = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_dev.csv')
dev = dev[dev.notnull().apply(all, axis=1)]
test = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_test.csv')
test = test[test.notnull().apply(all, axis=1)]

dev_prepared = pd.read_csv('../02_Extract_Subphrases/prepared_data/subphrase_vectors_dev.csv', sep=';')
dev_prepared = dev_prepared.drop(columns='Unnamed: 0')
dev = dev.set_index('pairID')
rel_pairIDs = dev_prepared.iloc[:,0]
y_hat = dev.loc[rel_pairIDs].gold_label
dev_prepared = dev_prepared.iloc[:,1:].to_numpy()

In [4]:
temp = [None, ] * 10
for i in range(1,11):
    temp[i-1] = pd.read_csv("prepared_data/GPT3_explanations" + str(i) + ".csv", sep=";")
gpt3_dev = pd.concat(temp).set_index("pairID")

In [5]:
np.mean(gpt3_dev.loc[rel_pairIDs].gold_standard_label == gpt3_dev.loc[rel_pairIDs].pred_label)

0.8069192751235584

In [6]:
f1_score(gpt3_dev.loc[rel_pairIDs].gold_standard_label, gpt3_dev.loc[rel_pairIDs].pred_label, average="macro")

0.7884484917636336

In [7]:
gpt3_dev.loc[rel_pairIDs].iloc[np.where(gpt3_dev.loc[rel_pairIDs].gold_standard_label == gpt3_dev.loc[rel_pairIDs].pred_label)]

,Unnamed: 0,gold_standard_explanation,gold_standard_label,prompts_instruct_GPT,pred_explanation,pred_label
pairID,,,,,,
4705552913.jpg#2r1e,1,Saying the two women are holding packages is a...,entailment,"Classify into entailment, neutral, or contradi...",Embracing is a form of holding.,entailment
4705552913.jpg#2r1c,2,In the first sentence there is an action of af...,contradiction,"Classify into entailment, neutral, or contradi...",Embracing and fighting are different actions.,contradiction
3948003394.jpg#1r1e,18,Both woman are doing a cartwheel.,entailment,"Classify into entailment, neutral, or contradi...",A woman doing a cartwheel is wearing a bikini ...,entailment
3948003394.jpg#1r1c,19,It is not feasible to be in the process of fix...,contradiction,"Classify into entailment, neutral, or contradi...",A woman doing a cartwheel in a bikini in the s...,contradiction
5436250638.jpg#4r1c,42,The subjects cannot perform surgery and be hav...,contradiction,"Classify into entailment, neutral, or contradi...",Two doctors performing surgery cannot be havin...,contradiction
...,...,...,...,...,...,...
6087966524.jpg#2r1e,9827,Two police officers can be rephrased as The Po...,entailment,"Classify into entailment, neutral, or contradi...",The premise states that the police officers ar...,entailment
3344632789.jpg#0r2c,9828,Performing an aerial skateboard trick and orde...,contradiction,"Classify into entailment, neutral, or contradi...",A man performing an aerial skateboard trick on...,contradiction
3344632789.jpg#0r1e,9831,Several people are people.,entailment,"Classify into entailment, neutral, or contradi...",The premise states that there are several peop...,entailment


In [14]:
for i in range(100):
    print(gpt3_dev.loc[rel_pairIDs].pred_explanation.iloc[i])

Two women embracing while holding to go packages implies that they just ate lunch and are now saying goodbye.
Embracing is a form of holding.
Embracing and fighting are different actions.
A woman doing a cartwheel is wearing a bikini in the sand next to the beach.
A woman doing a cartwheel in a bikini in the sand implies she is not fixing her home.
A woman can do a cartwheel and not fall on her head.
Two doctors performing surgery cannot be having lunch at the same time.
The premise states that two doctors are performing surgery, which is what the hypothesis states.
The premise does not specify the gender of the patient.
A dog cannot be jumping and swimming underwater at the same time.
A white dog with long hair jumps to catch a red and green toy is same as an animal is jumping to catch an object.
A white dog with long hair does not necessarily mean it is a chihuahua mix with long curly hair.
Kids are on a amusement ride is same as Kids ride their favorite amusement ride.
Kids cannot b

In [9]:
gpt3_dev.index[1235]

'4345676156.jpg#4r1e'